In [302]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from datetime import datetime
import pandasql as ps
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Import Tweets - https://www.kaggle.com/alaix14/bitcoin-tweets-20160101-to-20190329
#tweets_df = pd.read_csv('tweets.csv', error_bad_lines=False, 
#                        names=['user', 'fullname', 'timestamp', 'replies', 'likes', 'retweets', 'text'], 
#                        sep=';', header=2, nrows=10)
tweets_df = pd.read_csv('tweets.csv', sep=';')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,2,3,4,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
# Drop url, id, fullname
tweets_df = tweets_df.drop('url', 1)
tweets_df = tweets_df.drop('id', 1)
tweets_df = tweets_df.drop('fullname', 1)

In [12]:
# Drop all rows where text is null
tweets_df = tweets_df.dropna(axis=0, subset=['text'])
# Drop all rows where timestamp is null
tweets_df = tweets_df.dropna(axis=0, subset=['timestamp'])

In [18]:
# Change timestamp dtype
tweets_df['timestamp'] = tweets_df['timestamp'].apply(lambda x: datetime.strptime(x[:-3], '%Y-%m-%d %H:%M:%S'))

In [ ]:
tweets_df

In [19]:
import re

def remove_bad_patterns(text):
    """Remove html, latex, and newline characters from a string
    
    :param text: content as a string
    :return: cleaned text string
    """
    html_tag = re.compile('<.*?>')
    stripped_html = re.sub(html_tag, '', text)
    stripped_n = stripped_html.strip().replace('\n', '')
    return stripped_n

In [20]:
# Clean all the text data - remove tags/new lines
tweets_df['text'] = tweets_df['text'].apply(lambda x: remove_bad_patterns(x))

In [28]:
# Separate dfs by time
# March 22, 2017 ($1036.10) - June 10, 2017 ($2910.34): First big rise - period_one
# June 10, 2017 ($2910.34) - July 14, 2017 ($2109.37): First big fall - period_two
# Sept 14, 2017 ($3358.11) - Dec 16, 2017 ($19166.88): Massive peak in 2017 - period_three
# Dec 16, 2017 ($19166.88) - Feb 5, 2018 ($6332.37): Massive fall in 2017/18 - period_four
# Feb 5, 2018 ($6332.37) - Dec 15, 2018 ($3194.96): Intense volatility and uncertainty - period_five

In [158]:
period_one = tweets_df[(tweets_df['timestamp'] > '2017-03-22 00:00:00') & (tweets_df['timestamp'] < '2017-06-10 23:59:00')]
period_two = tweets_df[(tweets_df['timestamp'] > '2017-06-10 00:00:00') & (tweets_df['timestamp'] < '2017-07-14 23:59:00')]
period_three = tweets_df[(tweets_df['timestamp'] > '2017-07-14 00:00:00') & (tweets_df['timestamp'] < '2017-12-16 23:59:00')]
period_four = tweets_df[(tweets_df['timestamp'] > '2017-12-16 00:00:00') & (tweets_df['timestamp'] < '2018-02-05 23:59:00')]
period_five = tweets_df[(tweets_df['timestamp'] > '2018-02-05 00:00:00') & (tweets_df['timestamp'] < '2018-12-15 23:59:00')]

In [27]:
print(len(period_one))
print(len(period_two))
print(len(period_three))
print(len(period_four))
print(len(period_five))

31333
50264
783011
76104
2107050


In [159]:
# Only use tweets that have at least 10 likes, aka delete the losers
period_one_cleaned = period_one[(period_one['likes'] >= 10)]
period_two_cleaned = period_two[(period_two['likes'] >= 10)]
period_three_cleaned = period_three[(period_three['likes'] >= 10)]
period_four_cleaned = period_four[(period_four['likes'] >= 10)]
period_five_cleaned = period_five[(period_five['likes'] >= 10)]

In [54]:
print(len(period_one_cleaned))
print(len(period_two_cleaned))
print(len(period_three_cleaned))
print(len(period_four_cleaned))
print(len(period_five_cleaned))

349
872
12643
2135
37777


In [103]:
print(len(period_one_cleaned)/len(period_one))
print(len(period_two_cleaned)/len(period_two))
print(len(period_three_cleaned)/len(period_three))
print(len(period_four_cleaned)/len(period_four))
print(len(period_five_cleaned)/len(period_five))

0.011138416366131554
0.017348400445646985
0.016146644172304093
0.028053715967623252
0.017928857881872762


In [71]:
period_one_text = period_one_cleaned['text'].tolist()
period_two_text = period_two_cleaned['text'].tolist()
period_three_text = period_three_cleaned['text'].tolist()
period_four_text = period_four_cleaned['text'].tolist()
period_five_text = period_five_cleaned['text'].tolist()

In [72]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varunramakrishnan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/varunramakrishnan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [73]:
stopwords = set(stopwords.words('english')) 

In [79]:
# Add bitcoin specific words
stopwords.add('bitcoin')
stopwords.add('btc')
stopwords.add('http')
stopwords.add('https')
stopwords.add('r')
stopwords.add('www')
stopwords.add('.com')
stopwords.add('...')
stopwords.add('…')
stopwords.add('’')

In [80]:
import string

def tokenize_content(content):
  """returns tokenized string

  :param content: text string
  :return: tokenized text/list of words
  """
  list_of_digits = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
  # Create tokens, make all lowercase, remove all stopwords
  tokens = nltk.word_tokenize(content)
  tokens = [token.lower() for token in tokens]
  tokens_cleaned = [token for token in tokens if token not in stopwords and token not in string.punctuation
                    and token[0] not in string.punctuation]

  # Extra operations for word cloud - remove numbers
  tokens_no_numbers = [token for token in tokens_cleaned if not any(c.isdigit() for c in token)]
  # Take out punctuation in the middle
  final_tokens = [token for token in tokens_no_numbers if not any(p in token for p in string.punctuation)]
  return final_tokens

In [94]:
all_highest = [tokenize_content(item) for item in period_one_text]
highest_tokens = [item for sublist in all_highest for item in sublist]

In [95]:
from collections import Counter 

highest_counter = Counter(highest_tokens) 
highest_most_common_all = highest_counter.most_common
highest_most_common = highest_counter.most_common(20) 

In [96]:
highest_most_common

[('blockchain', 52),
 ('price', 26),
 ('usd', 24),
 ('crypto', 21),
 ('amp', 21),
 ('new', 17),
 ('fintech', 17),
 ('may', 16),
 ('latest', 15),
 ('ico', 14),
 ('today', 14),
 ('eth', 14),
 ('mining', 14),
 ('time', 13),
 ('ethereum', 13),
 ('gmt', 13),
 ('pm', 12),
 ('buy', 12),
 ('april', 12),
 ('index', 12)]

In [144]:
# Sentiment Analysis using VADER (given social medida focus)
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/varunramakrishnan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [160]:
# Period 1 through 5 Sentiment Scores
period_one_cleaned['neg'] = period_one_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neg'], axis=1)
period_one_cleaned['neu'] = period_one_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neu'], axis=1)
period_one_cleaned['pos'] = period_one_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['pos'], axis=1)
period_one_cleaned['compound'] = period_one_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['compound'], axis=1)

period_two_cleaned['neg'] = period_two_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neg'], axis=1)
period_two_cleaned['neu'] = period_two_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neu'], axis=1)
period_two_cleaned['pos'] = period_two_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['pos'], axis=1)
period_two_cleaned['compound'] = period_two_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['compound'], axis=1)

period_three_cleaned['neg'] = period_three_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neg'], axis=1)
period_three_cleaned['neu'] = period_three_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neu'], axis=1)
period_three_cleaned['pos'] = period_three_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['pos'], axis=1)
period_three_cleaned['compound'] = period_three_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['compound'], axis=1)

period_four_cleaned['neg'] = period_four_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neg'], axis=1)
period_four_cleaned['neu'] = period_four_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neu'], axis=1)
period_four_cleaned['pos'] = period_four_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['pos'], axis=1)
period_four_cleaned['compound'] = period_four_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['compound'], axis=1)

In [161]:
period_five_cleaned['neg'] = period_five_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neg'], axis=1)
period_five_cleaned['neu'] = period_five_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['neu'], axis=1)
period_five_cleaned['pos'] = period_five_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['pos'], axis=1)
period_five_cleaned['compound'] = period_five_cleaned.apply(lambda x: sid.polarity_scores(x['text'])['compound'], axis=1)

In [163]:
period_one_neg = period_one_cleaned['neg'].mean()
period_one_neu = period_one_cleaned['neu'].mean()
period_one_pos = period_one_cleaned['pos'].mean()
period_one_compound = period_one_cleaned['compound'].mean()

period_two_neg = period_two_cleaned['neg'].mean()
period_two_neu = period_two_cleaned['neu'].mean()
period_two_pos = period_two_cleaned['pos'].mean()
period_two_compound = period_two_cleaned['compound'].mean()

period_three_neg = period_three_cleaned['neg'].mean()
period_three_neu = period_three_cleaned['neu'].mean()
period_three_pos = period_three_cleaned['pos'].mean()
period_three_compound = period_three_cleaned['compound'].mean()

period_four_neg = period_four_cleaned['neg'].mean()
period_four_neu = period_four_cleaned['neu'].mean()
period_four_pos = period_four_cleaned['pos'].mean()
period_four_compound = period_four_cleaned['compound'].mean()

period_five_neg = period_five_cleaned['neg'].mean()
period_five_neu = period_five_cleaned['neu'].mean()
period_five_pos = period_five_cleaned['pos'].mean()
period_five_compound = period_five_cleaned['compound'].mean()

In [164]:
print("Period One")
print(period_one_neg)
print(period_one_neu)
print(period_one_pos)
print(period_one_compound)

print("Period Two")
print(period_two_neg)
print(period_two_neu)
print(period_two_pos)
print(period_two_compound)

print("Period Three")
print(period_three_neg)
print(period_three_neu)
print(period_three_pos)
print(period_three_compound)

print("Period Four")
print(period_four_neg)
print(period_four_neu)
print(period_four_pos)
print(period_four_compound)

print("Period Five")
print(period_five_neg)
print(period_five_neu)
print(period_five_pos)
print(period_five_compound)

Period One
0.029315186246418337
0.8873237822349573
0.08335530085959883
0.12711232091690544
Period Two
0.04455045871559632
0.887211009174312
0.06824082568807344
0.05573463302752292
Period Three
0.040174958475045465
0.8857470537056052
0.07399564976666936
0.09981891165071592
Period Four
0.030654332552693196
0.8988562060889944
0.07049227166276352
0.1333743325526934
Period Five
0.03279005744235934
0.8880077295709025
0.07920173650633962
0.1479023718135382


In [102]:
# Sentiment Analysis using FastText
import fasttext
# Omit because data labelling/supervised learning is not appropriate

In [282]:
# Create master sentiment CSV to compare sentiment/Twitter activity with price
# Start by loading Bitcoin price data
prices_df = pd.read_csv('prices.csv')

In [283]:
# Convert timestamp to datetime format for join
prices_df['Timestamp'] = prices_df['Timestamp'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
# Change type of column to datetime
prices_df['Timestamp']= pd.to_datetime(prices_df['Timestamp'])

In [285]:
# Basically, we have to calculate sentiment for everything in the previous dataframe
# Also, do group bys to organize data by hour

In [288]:
# Truncate datetime to hour
tweets_df['timestamp'] = tweets_df['timestamp'].apply(lambda x: x.replace(microsecond=0, second=0, minute=0))
# Calculate compound score
tweets_df['Compound_Score'] = tweets_df.apply(lambda x: sid.polarity_scores(x['text'])['compound'], axis=1)

In [289]:
# Drop unneeded columns
tweets_df = tweets_df.drop('replies', 1)
tweets_df = tweets_df.drop('likes', 1)
tweets_df = tweets_df.drop('retweets', 1)
tweets_df = tweets_df.drop('text', 1)
tweets_df = tweets_df.drop('user', 1)

# Add number of Tweets
tweets_df['Total_Num_Tweets'] = tweets_df.groupby(['timestamp'])['Compound_Score'].transform('count')

# Group by timestamp 
tweets_df = tweets_df.groupby(by=["timestamp"], as_index=False).mean()

In [317]:
# Join the two dataframes on timestamp!
# Left join on price 
sentiments_df = prices_df.merge(tweets_df, left_on='Timestamp', right_on='timestamp', how='left')
# Drop timestamp
sentiments_df = sentiments_df.drop('timestamp', 1)
sentiments_df = sentiments_df.drop('Weighted_Price', 1)

In [320]:
# Add number of negative, positive, and neutral tweets using sql
query = '''SELECT Timestamp, Open, High, Low, Close, [Volume (BTC)], [Volume (Currency)], Compound_Score, Total_Num_Tweets
,(CASE WHEN Compound_Score < 0 THEN 1 ELSE 0 END) AS Num_Negatives
,(CASE WHEN Compound_Score > 0 THEN 1 ELSE 0 END) AS Num_Positives
,(CASE WHEN Compound_Score = 0 THEN 1 ELSE 0 END) AS Num_Neutrals
FROM sentiments_df
'''

output_df = ps.sqldf(query, locals())

In [323]:
# Export as sentiments.csv
output_df.to_csv('sentiments.csv', index=False)